In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# using all faculty pages at once
FacultyLinks=[
    'http://pwr.nu.edu.pk/cs-faculty/',
    'http://pwr.nu.edu.pk/ee-faculty/',
    'http://pwr.nu.edu.pk/sh-faculty/'
]

In [3]:
id=[]
name=[]
image=[]
designation=[]
HECApproved=[]
department=[]
email=[]
ext=[]
highestEducation=[]

In [ ]:
# extracting all data from the faculty personal profile
for link in FacultyLinks:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')

    teachers = soup.findAll('a', class_='text-theme-color-2')

    for teacher in teachers:
        teacherLink = 'http://pwr.nu.edu.pk/' + teacher['href']
        teacher_id = teacherLink.split('=')[-1]
        id.append(teacher_id)

        response2 = requests.get(teacherLink)
        soup2 = BeautifulSoup(response2.content, 'html.parser')

        name.append(soup2.find('h3', class_='name font-24 mt-0 mb-0').text.strip() if soup2.find('h3', class_='name font-24 mt-0 mb-0') else "N/A")
        designation.append(soup2.find('h5', class_='mt-5 text-theme-color-2').text.strip() if soup2.find('h5', class_='mt-5 text-theme-color-2') else "N/A")
        
        department_info = soup2.find('h2', class_='title text-white')
        department.append(department_info.text.strip()[:-8] if department_info else "N/A")

        hec_status = soup2.find('h5', style='color: green')
        HECApproved.append(hec_status.text.strip() != '' if hec_status else False)

        education_list = soup2.find('ul', class_='list')
        highest_edu = education_list.find('li').text.strip() if education_list and education_list.find('li') else "N/A"
        highestEducation.append(highest_edu)

        contact_info = soup2.find('h5', string=lambda text: text and "Ext." in text)
        ext.append(int(contact_info.text.split()[-1]) if contact_info else None)

        email_info = soup2.find('a', href=lambda href: href and "mailto" in href)
        email.append(email_info.text.strip() if email_info else "N/A")

        img_tag = soup2.find('div', class_='thumb')
        img_src = img_tag.find('img')['src'] if img_tag and img_tag.find('img') else None
        image.append('http://pwr.nu.edu.pk' + img_src if img_src else "N/A")

C:\Users\f-jaf\AppData\Local\Temp\ipykernel_12628\2983645856.py:5: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  teachers = soup.findAll('a', class_='text-theme-color-2')


In [8]:
df = pd.DataFrame({
    "ID": id,
    "Name": name,
    "Designation": designation,
    "Department": department,
    "HEC Approved": HECApproved,
    "Highest Education": highestEducation,
    "Extension": ext,
    "Email": email,
    "Image URL": image
})

In [ ]:
#checking first 5 rows
df.head()

,ID,Name,Designation,Department,HEC Approved,Highest Education,Extension,Email,Image URL
0,3704,Fazl-e-Basit,Assistant Professor,Department of Computer,False,"M.S(Computer Science), NUST, Rawalpindi, Pakis...",None,fazl.basit@nu.edu.pk,http://pwr.nu.edu.pkimages/faculty/profile3704...
1,9237,Dr. Omar Usman Khan,Professor & Director,Department of Computer,True,"Ph.D. (Computer And Control Engineering), Poli...",None,omar.khan@nu.edu.pk,http://pwr.nu.edu.pkimages/faculty/profile9237...
2,9315,Dr. Hafeez Ur Rehman,Associate Professor,Department of Computer,True,"Ph.D. in Computer and Control Engineering, Pol...",None,hafeez.urrehman@nu.edu.pk,http://pwr.nu.edu.pkimages/faculty/profile9315...
3,9482,Dr. Nouman Azam,Associate Professor,Department of Computer,True,"Ph.D. (CS), University of Regina, Canada (2014)",None,nouman.azam@nu.edu.pk,http://pwr.nu.edu.pkimages/faculty/profile9482...
4,6833,Dr. Hafeez Anwar,Associate Professor,Department of Computer,False,"PhD (Informatics), Vienna University of Techno...",None,hafeez.anwar@nu.edu.pk,http://pwr.nu.edu.pkimages/faculty/profile6833...


In [ ]:
# saving to csv
df.to_csv("pwr.csv", index=False)